In [2]:
!pip install faiss-gpu-cu12 --extra-index-url=https://pypi.nvidia.com --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 220.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import faiss  # faiss 라이브러리 호출

In [5]:
dimension = 128    # 각 벡터의 차원
n = 200    # 벡터 수
np.random.seed(1)
db_vectors = np.random.random((n, dimension)).astype('float32') #(200 * 128) 벡터 행렬

In [6]:
nlist = 5  # 클러스터(셀이라고도 표현) 수
quantiser = faiss.IndexFlatL2(dimension)  #quantizer를 활용해서 클러스터 생성
index = faiss.IndexIVFFlat(quantiser, dimension, nlist,   faiss.METRIC_L2)  #Inverted File 만들기#langchain에서 쓰는 인덱스x 임의로 만들어줌
#index = faiss.IndexPQ(D, m, nbits)
#index = faiss.IndexIVFPQ(vecs, D, nlist, m, nbits)

In [7]:
print(index.is_trained)   # False
index.train(db_vectors)  # 데이터베이스 벡터에 대한 훈련
print(index.ntotal)
index.add(db_vectors)   # 벡터를 추가하고 인덱스를 업데이트
print(index.is_trained)  # True
print(index.ntotal)

False
0
True
200


In [8]:
#nprobe = 2  # 가장 유사한 클러스터 2개 찾기
n_query = 10  # 10개의 쿼리 벡터
k = 3  # 가장 가까운 이웃 3개를 반환
np.random.seed(1)  # 일관된 결과를 얻기 위해 시드 설정
query_vectors = np.random.random((n_query, dimension)).astype('float32') #쿼리 벡터 생성
distances, indices = index.search(query_vectors, k) #검색 수행


In [9]:
faiss.write_index(index,"vector.index")  # 인덱스를 디스크에 저장
index = faiss.read_index("vector.index")  # 인덱스를 로드